In [3]:
seed = 42
proj_id = 'cmat-315920'
root_path  = '/home/jupyter'
code_path = root_path + '/MathVGerrmandering_CMAT_2021/src/'
for f in ['Core', 'Gerry', 'Variable', 'Crosswalks', 'Assignments', 'Shapes',
          'Census', 'Elections', 'Votes', 'Combined']:
    %run -i {code_path+f+'.py'}

refresh_tbl = (
#     'crosswalks',
#     'assignments',
#     'shapes',
#     'census',
#     'elections',
#     'votes_all',
#     'votes_hl',
#     'combined',
)

refresh_all = (
#     'crosswalks',
#     'assignments',
#     'shapes',
#     'census',
#     'elections',
#     'votes_all',
#     'votes_hl',
#     'combined',
)

g = Gerry(abbr='TX', level='cnty',
          election_filters=(
            "office='USSen' and race='general'",
            "office='President' and race='general'",
            "office like 'USRep%' and race='general'"),
          refresh_tbl=refresh_tbl, refresh_all=refresh_all
         )
g.get_data()

Get crosswalks TX 2010 tabblock              ... tabblock table exists ... success
Get assignments TX 2020 tabblock             ... tabblock table exists ... success
Get shapes TX 2020 tabblock                  ... tabblock table exists ... success
Get census TX 2010 tabblock                  ... tabblock table exists ... success
Get elections TX 2020 tabblock               ... tabblock table exists ... success
Get votes_all TX 2020 tabblock               ... tabblock table exists ... success
Get votes_hl TX 2020 tabblock                ... tabblock table exists ... success
Get combined TX 2020 cnty                    ... raw table exists ... creating table ... success
